<a href="https://colab.research.google.com/github/whs1111/bert-version/blob/master/bert1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive

drive.mount('/content/gdrive')

In [19]:
cd My Drive/Colab_Notebooks/code

/content/gdrive/My Drive/Colab_Notebooks/code


In [20]:
ls

bert-version/


In [22]:
!python

Python 3.6.7 (default, Oct 22 2018, 11:32:17) 
[GCC 8.2.0] on linux
Type "help", "copyright", "credits" or "license" for more information.
>>> 
KeyboardInterrupt
>>> 
KeyboardInterrupt
>>> 
KeyboardInterrupt
>>> ^C


In [0]:
#@title
!git clone https://github.com/NVIDIA/apex.git
%cd apex
!pip install -v --no-cache-dir --global-option="--cpp_ext" --global-option="--cuda_ext" .
%cd ..

import torch
import apex
from fastai.text import *
import datetime
run_start_time = datetime.datetime.today().strftime('%Y-%m-%d_%H-%M-%S')

LOG_PATH=Path('logs/')  
MODEL_PATH=Path('models/') 

if not LOG_PATH.exists():
  LOG_PATH.mkdir()
import logging

args = {
    "run_text": "my_test",
    "max_seq_length": 30,
    "do_lower_case": True,
    "train_batch_size": 16,
    "learning_rate": 6e-5,
    "num_train_epochs": 12.0,
    "warmup_proportion": 0.002,
    "local_rank": -1,
    "gradient_accumulation_steps": 1,
    "fp16": True,
    "loss_scale": 128
}

logfile = str(LOG_PATH/'log-{}-{}.txt'.format(run_start_time, args["run_text"]))

logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(name)s -   %(message)s',
    datefmt='%m/%d/%Y %H:%M:%S',
    handlers=[
        logging.FileHandler(logfile),
        logging.StreamHandler(sys.stdout)
    ])

logger = logging.getLogger()

device = torch.device('cuda')

if torch.cuda.device_count() > 1:
    multi_gpu = True
else:
    multi_gpu = False

Cloning into 'apex'...
remote: Enumerating objects: 86, done.
remote: Counting objects: 100% (86/86), done.
remote: Compressing objects: 100% (60/60), done.
remote: Total 4722 (delta 42), reused 54 (delta 26), pack-reused 4636
Receiving objects: 100% (4722/4722), 8.75 MiB | 25.16 MiB/s, done.
Resolving deltas: 100% (3054/3054), done.
/content/apex
/usr/local/lib/python3.6/dist-packages/pip/_internal/commands/install.py:244: UserWarning: Disabling all use of wheels due to the use of --build-options / --global-options / --install-options.
  cmdoptions.check_install_build_global(options)
Created temporary directory: /tmp/pip-ephem-wheel-cache-8yy_cnjb
Created temporary directory: /tmp/pip-req-tracker-t1zfn7nz
Created requirements tracker '/tmp/pip-req-tracker-t1zfn7nz'
Created temporary directory: /tmp/pip-install-aox20ntb
Processing /content/apex
  Created temporary directory: /tmp/pip-req-build-dltx93k0
  Added file:///content/apex to build tracker '/tmp/pip-req-tracker-t1zfn7nz'
    Ru

In [0]:
!git clone git@github.com:whs1111/bert-version.git

Cloning into 'bert-version'...
Host key verification failed.
fatal: Could not read from remote repository.

Please make sure you have the correct access rights
and the repository exists.


In [23]:
!pip install fast-bert

     |████████████████████████████████| 133kB 9.6MB/s 
     |████████████████████████████████| 655kB 52.6MB/s 
  Stored in directory: /root/.cache/pip/wheels/35/e4/80/abf3b33ba89cf65cd262af8a22a5a999cc28fbfabea6b38473
Successfully built regex


In [0]:
from fast_bert.data import *
from fast_bert.learner import *
from fast_bert.metrics import *
from pytorch_pretrained_bert.tokenization import BertTokenizer

In [26]:
DATA_PATH = Path('bert-version/src/DATA/')     
LABEL_PATH = Path('bert-version/src/DATA/')  

BERT_PRETRAINED_MODEL = "bert-base-uncased"

args["do_lower_case"] = True
args["train_batch_size"] = 16
args["learning_rate"] = 6e-5
args["max_seq_length"] = 140
args["fp16"] = True
tokenizer = BertTokenizer.from_pretrained(BERT_PRETRAINED_MODEL, 
                                          do_lower_case=args['do_lower_case'])
label_cols = ["GoodsServices",
        "SearchAndRescue",
        "InformationWanted",
        "Volunteer",
        "FundRaising",
        "Donations",
        "MovePeople",
        "FirstPartyObservation",
        "ThirdPartyObservation",
        "Weather",
        "EmergingThreats",
        "NewSubEvent",
        "MultimediaShare",
        "ServiceAvailable",
        "Factoid",
        "Official",
        "CleanUp",
        "Hashtags",
        "ContextualInformation",
        "News",
        "Advice",
        "Sentiment",
        "Discussion",
        "Irrelevant",
        "OriginalEvent"]
databunch = BertDataBunch(DATA_PATH, LABEL_PATH, tokenizer, train_file='train.csv', val_file='valid.csv',
                          test_data='test.csv', label_file="labels.csv",
                          text_col="text", label_col=label_cols,
                          bs=args['train_batch_size'], maxlen=args['max_seq_length'], 
                          multi_gpu=multi_gpu, multi_label=True)
metrics = [{'name': 'accuracy', 'function': accuracy_multilabel}]
logger.info(args)
learner = BertLearner.from_pretrained_model(databunch, BERT_PRETRAINED_MODEL, metrics, device, logger, 
                                            is_fp16=args['fp16'], loss_scale=args['loss_scale'], 
                                            multi_gpu=multi_gpu,  multi_label=True)
learner.fit(4, lr=args['learning_rate'], schedule_type="warmup_linear")

06/18/2019 06:44:10 - INFO - pytorch_pretrained_bert.tokenization -   loading vocabulary file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-vocab.txt from cache at /root/.pytorch_pretrained_bert/26bc1ad6c0ac742e9b52263248f6d0f00068293b33709fae12320c0e35ccfbbb.542ce4285a40d23a559526243235df47c5f75c197f04f37d1a0c124c32c9a084
Cannot tokenise item 3428, Text:nan
Cannot tokenise item 3601, Text:nan
Cannot tokenise item 5765, Text:nan
Cannot tokenise item 14143, Text:nan
Cannot tokenise item 333, Text:nan
Cannot tokenise item 1806, Text:nan
Cannot tokenise item 3214, Text:nan
06/18/2019 06:44:31 - INFO - root -   {'run_text': 'my_test', 'max_seq_length': 140, 'do_lower_case': True, 'train_batch_size': 16, 'learning_rate': 6e-05, 'num_train_epochs': 12.0, 'warmup_proportion': 0.002, 'local_rank': -1, 'gradient_accumulation_steps': 1, 'fp16': True, 'loss_scale': 128}
06/18/2019 06:44:31 - INFO - pytorch_pretrained_bert.file_utils -   https://s3.amazonaws.com/models.hugg

100%|██████████| 407873900/407873900 [00:05<00:00, 68035686.44B/s]

06/18/2019 06:44:38 - INFO - pytorch_pretrained_bert.file_utils -   copying /tmp/tmpzrjfyy8z to cache at /root/.pytorch_pretrained_bert/9c41111e2de84547a463fd39217199738d1e3deb72d4fec4399e6e241983c6f0.ae3cef932725ca7a30cdcb93fc6e09150a55e2a130ec7af63975a16c153ae2ba


06/18/2019 06:44:39 - INFO - pytorch_pretrained_bert.file_utils -   creating metadata file for /root/.pytorch_pretrained_bert/9c41111e2de84547a463fd39217199738d1e3deb72d4fec4399e6e241983c6f0.ae3cef932725ca7a30cdcb93fc6e09150a55e2a130ec7af63975a16c153ae2ba
06/18/2019 06:44:39 - INFO - pytorch_pretrained_bert.file_utils -   removing temp file /tmp/tmpzrjfyy8z
06/18/2019 06:44:39 - INFO - pytorch_pretrained_bert.modeling -   loading archive file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased.tar.gz from cache at /root/.pytorch_pretrained_bert/9c41111e2de84547a463fd39217199738d1e3deb72d4fec4399e6e241983c6f0.ae3cef932725ca7a30cdcb93fc6e09150a55e2a130ec7af63975a16c153ae2ba
06/18/2019 06:44:39 - INFO - pytorch_pretrained_bert.modeling -   extracting archive file /root/.pytorch_pretrained_bert/9c41111e2de84547a463fd39217199738d1e3deb72d4fec4399e6e241983c6f0.ae3cef932725ca7a30cdcb93fc6e09150a55e2a130ec7af63975a16c153ae2ba to temp dir /tmp/tmp1aid2ewu
06/18/2019 06:44:44 -

06/18/2019 06:47:31 - INFO - root -   Loss after epoch 0 - 0.21507104440352973
06/18/2019 06:47:31 - INFO - root -   Running evaluation


06/18/2019 06:47:57 - INFO - root -   Eval results:
06/18/2019 06:47:57 - INFO - root -     eval_loss = 0.17679741753472222
06/18/2019 06:47:57 - INFO - root -     metrics = {'accuracy': 0.28055555555555556}
06/18/2019 06:47:57 - INFO - root -   --------------------------------------------------------------------------------
06/18/2019 06:50:32 - INFO - root -   Loss after epoch 1 - 0.1571281416907179
06/18/2019 06:50:32 - INFO - root -   Running evaluation


06/18/2019 06:50:58 - INFO - root -   Eval results:
06/18/2019 06:50:58 - INFO - root -     eval_loss = 0.17458821614583334
06/18/2019 06:50:58 - INFO - root -     metrics = {'accuracy': 0.34194444444444444}
06/18/2019 06:50:58 - INFO - root -   --------------------------------------------------------------------------------
06/18/2019 06:53:34 - INFO - root -   Loss after epoch 2 - 0.1301938196656051
06/18/2019 06:53:34 - INFO - root -   Running evaluation


06/18/2019 06:53:59 - INFO - root -   Eval results:
06/18/2019 06:53:59 - INFO - root -     eval_loss = 0.17703179253472223
06/18/2019 06:53:59 - INFO - root -     metrics = {'accuracy': 0.35555555555555557}
06/18/2019 06:53:59 - INFO - root -   --------------------------------------------------------------------------------
06/18/2019 06:56:35 - INFO - root -   Loss after epoch 3 - 0.1079495180943969
06/18/2019 06:56:35 - INFO - root -   Running evaluation


06/18/2019 06:57:01 - INFO - root -   Eval results:
06/18/2019 06:57:01 - INFO - root -     eval_loss = 0.1801193576388889
06/18/2019 06:57:01 - INFO - root -     metrics = {'accuracy': 0.38027777777777777}
06/18/2019 06:57:01 - INFO - root -   --------------------------------------------------------------------------------
